01/12/2017

### Andrés Méndez

# Econ 388 E - Fall 2017
## Assignment 3: Single Agent Dynamic Programming / Estimation


Consider a capital replacement problem similar to that in Rust (1987). Firms each use one machine to produce output in each period. These machines age, becoming more likely to breakdown, and in each time period the firms have the option of replacing the machines. Let $a_t$ be the age of your machine at time $t$ and let the expected current period profits from using a machine of age $a_t$ be given by:

\begin{equation}
\Pi(a_t, i_t, \varepsilon_{0t}, \varepsilon_{1t})=
\begin{cases}
&\theta_1 a_t + \varepsilon_{0t} &\quad \text{if $i_t =0$}\\
&R + \varepsilon_{1t} &\quad \text{if $i_t =1$}
\end{cases}
\end{equation}

where $i_t = 1$ if the firm decides to replace the machine at $t$, $R$ is the net cost of a new machine, and the $\varepsilon_{t}$'s are time specific shocks to the utilities from replacing and not replacing. Assume that these $\varepsilon_{t}$'s are i.i.d. logit errors.

Lets assume a very simple state evolution equation:

\begin{equation}
a_{t+1}=
\begin{cases}
&\min\{5, a_t+1\} &\quad \text{if $i_t =0$}\\
&1 &\quad \text{if $i_t =1$}
\end{cases}
\end{equation}

In words, if the firm decides not to replace, the machine ages by one year (up to a maximum of 5 years - after 5 years machines don't age). If the firm replaces in the current year, the age next year is 1. Note that there are thus only 5 possible values of $a_t$ - 1,2,3,4, and 5.

### Question 1

Write down the dynamic programming problem for a firm maximizing the PDV of future profits (assume an $\infty$ horizon).

**Answer:**

The stochastic process governing $\{i_t, a_t\}$ is the solution to the following  *regenerative optimal stopping* problem:

\begin{equation}
V(a_t; \theta) = \max_{\Phi} E\bigg[\sum_{j=t}^\infty \beta^{j-t} \Pi(a_j, i_j, \varepsilon_{0j}, \varepsilon_{1j}; \theta) \bigg| a_t, \Phi; \theta\bigg],
\end{equation}

where expectation is over future $a_t$'s, and the $\max$ is taken over future choices $i_{t+1}, \dots, i_{\infty}$. However, because future $a$'s are observed before choosing future $i$'s, this is a functional $\Phi$ mapping future stares into future choices.

Under regularity conditions, $V(a_t)$ satisfies the following Bellman equation:

\begin{equation}
V(a_t; \theta) = \max_{i_t} [\Pi(a_j, i_j, \varepsilon_{0j}, \varepsilon_{1j}; \theta)+\beta [V(a_{t+1}; \theta)|a_t, i_t, \varepsilon_{0t}, \varepsilon_{1t};\theta]]
\end{equation}

### Question 2

What are the differences between this $\Pi(a_t, i_t, \varepsilon_{0t}, \varepsilon_{1t})$ and the profit function in the class notes on Rust? What happened to $c(0; \theta)$?

**Answer:**

The cost of operating a machine with age $a_t$ is assumed to be linear in age, i.e. $c(a_t; \theta)= \theta_1 a_t$. This implies that $c(0; \theta)=0$.

### Question 3

On the computer, write a procedure that solves this dynamic programming problem given values of the parameters $(\underline{\theta_1}, R)$. Assuem that $\beta = 0.9$. Use the "alternative-specific" value function method in the class, i,e, definte $\bar{V}_0(a_t)$ and $\bar{V}_1(a_t)$ -you should end up with equations looking something like

\begin{equation}
\begin{split}
&\bar{V}_0(a_t) = \theta_1 a_t + \beta E \big[\max\big\{\bar{V}_0(a_{t+1})+\varepsilon_{0t+1},\bar{V}_1(a_{t+1})+\varepsilon_{1t+1}  \big\} \big]\\
&\bar{V}_1(a_t) = R + \beta E \big[\max\big\{\bar{V}_0(a_{t+1})+\varepsilon_{0t+1},\bar{V}_1(a_{t+1})+\varepsilon_{1t+1}  \big\} \big]
\end{split}
\end{equation}

and do the contraction mapping on these two 5-vectors. Iterate the contraction mapping until the $\bar{V}$ functions don't change very much. Remember that given the logit error assumption there is an analytic solution to the
expectation of the max in these equations (and Euler's constant $\approx .5772$).

**Answer:**

Using te logit error assumption, we can integrate analytically over $\varepsilon_{0t+1}$ and $\varepsilon_{1t+1}$:

\begin{equation}
\begin{split}
&\bar{V}_0(a_t) = \theta_1 a_t + \beta E_{a_{t+1}} \big[0.5772 + \ln[e^{\bar{V}_0(a_{t+1})}+e^{\bar{V}_1(a_{t+1})}]\big]\\
&\bar{V}_1(a_t) = R + \beta E_{a_{t+1}} \big[0.5772 + \ln[e^{\bar{V}_0(a_{t+1})}+e^{\bar{V}_1(a_{t+1})}]\big].
\end{split}
\end{equation}


Moreover, the evolution of the state is  deterministic. Therefore, we rexpress the above equations as:

\begin{equation}
\begin{split}
&\bar{V}_0(a_t) = \theta_1 a_t + \beta  \big[0.5772 + \ln[e^{\bar{V}_0(\min\{5, a_t+1\})}+e^{\bar{V}_1(\min\{5, a_t+1\})}]\big]\\
&\bar{V}_1(a_t) = R + \beta  \big[0.5772 + \ln[e^{\bar{V}_0(1)}+e^{\bar{V}_1(1)}]\big].
\end{split}
\end{equation}


In [481]:
import nbconvert
import numpy as np
from numpy import linalg as LA
import scipy.stats 
from scipy.optimize import minimize
import pandas as pd

In [482]:
def contraction(Vn, beta, theta1, R):
    Vnplusone=np.zeros((10, 1))
    # The first 5 rows correspond to V_0. Ordered by the possible values of a_t: 1,2,3,4,5.
    Vnplusone[0] = theta1*1 + beta*(0.5772+np.log(np.exp(Vn[1])+np.exp(Vn[6])))
    Vnplusone[1] = theta1*2 + beta*(0.5772+np.log(np.exp(Vn[2])+np.exp(Vn[7])))
    Vnplusone[2] = theta1*3 + beta*(0.5772+np.log(np.exp(Vn[3])+np.exp(Vn[8])))
    Vnplusone[3] = theta1*4 + beta*(0.5772+np.log(np.exp(Vn[4])+np.exp(Vn[9])))
    Vnplusone[4] = theta1*5 + beta*(0.5772+np.log(np.exp(Vn[4])+np.exp(Vn[9])))
    #The last 5 rows correspond to V_1
    Vnplusone[5] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[6] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[7] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[8] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[9] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    return Vnplusone

In [483]:
# Parameter values:
beta = 0.9
theta1 = -1
R = -3

# Initial value for iterations:
Vn = np.ones((10, 1))

# Checking the contraction function:
#contraction(Vn, beta, theta1, R)

In [484]:
# Finding the fixed point:
count = 1
tres = 100
while (tres > 0.001):
    Vnplusone = contraction(Vn, beta, theta1, R)
    tres = LA.norm(Vnplusone-Vn)
    Vn = Vnplusone
    #print('Iteration:', count )
    #print('Threshold:', tres )
    #print('Vn:', Vn )
    count = count + 1

In [485]:
#Value function:
Vn

array([[-10.16653272],
       [-11.51456319],
       [-12.65497392],
       [-13.70797396],
       [-14.70797396],
       [-11.40007522],
       [-11.40007522],
       [-11.40007522],
       [-11.40007522],
       [-11.40007522]])



### Question 4

Solve the model for the parameters $(\theta_1 = -1, R = -3)$ . Suppose $a_t = 2$. Will the firm replace its machine in period $t$? Oops, that was a trick question - for what value of $\varepsilon_{0t}- \varepsilon_{1t}$ is the firm indifferent
between replacing its machine or not? What is the probability (to an econometrician who doesn't observe the $\varepsilon$'s) that this firm will replace its machine? What is the PDV of future profits for a firm at state
$\{a_t = 4,\varepsilon_{0t} = 1,\varepsilon_{1t} = -1.5\}$? (the constant term has been normalized so it is OK if this PDV is <0)


**Answer:**

* Note that:

\begin{equation}
V(a_t, \epsilon_t; \theta) =  \max_{i_t} [\bar{V}_0(a_t; \theta) + \epsilon_{0t}, \bar{V}_1(a_t; \theta) + \epsilon_{1t} ]
\end{equation}

In [486]:
# Present disocunted value of not replacing at age 2, net of epsilon_0t:
Vn[1]

array([-11.51456319])

In [487]:
# Present disocunted value of replacing at age 2, net of epsilon_1t:
Vn[6]

array([-11.40007522])

Therefore, for the parameters $(\theta_1 = -1, R = -3)$:

\begin{equation}
V(2, \epsilon_t; \theta) =  \max_{i_t} [-11.51456319 + \epsilon_{0t}, -11.40007522+ \epsilon_{1t} ].
\end{equation}

It follows that  the firm is indifferent between rplacing its machinge or not when:

\begin{equation}
-11.51456319 + \epsilon_{0t}= -11.40007522+ \epsilon_{1t},
\end{equation}

or

\begin{equation}
\epsilon_{0t}-\epsilon_{1t}= 11.51 -11.40 = 0.11.
\end{equation}


* The probability (to an econometrician who doesn't observe the $\varepsilon$'s) that this firm replace its machine is given by

\begin{equation}
\begin{split}
&Pr\big(-11.51456319 + \epsilon_{0t} \le -11.40007522+ \epsilon_{1t}\big| a_t=2\big)\\
&=Pr\big(\epsilon_{0t}-\epsilon_{1t} \le 0.11\big|a_t=2\big)= \frac{e^{0.11}}{1+e^{0.11}}= 0.53
\end{split}
\end{equation}

* The PDV of future profirs for a firm at state $\{a_t = 4,\varepsilon_{0t} = 1,\varepsilon_{1t} = -1.5\}$ is given by:

\begin{equation}
V(4,\varepsilon_{0t} = 1;\varepsilon_{1t} = -1.5; \theta) =  \max_{i_t} [-11.51456319 + 1, -11.40007522 -1.5 ]
\end{equation}

In [488]:
# Present disocunted value of not replacing at age 4:
Vn[3]+1

array([-12.70797396])

In [489]:
# Present disocunted value of replacing at age 4,:
Vn[8]-1.5

array([-12.90007522])

Therefore, it is optimal for the firm not to replace, and the PDV is

\begin{equation}
V(4,\varepsilon_{0t} = 1;\varepsilon_{1t} = -1.5; \theta) = -12.70797396
\end{equation}


### Question 5
There is a simulated dataset on Canvas, "data.asc". This dataset has just two columns - $a_t$ (first column) and $i_t$ (second column). Consider this as cross-sectional data - i.e. there is only one data point per firm. Estimate $(\theta_1, R)$ using maximum likelihood. Your ML function evaluation should look something like this:

  
**a)** Start with arbitrary $(\theta_1, R)$

**b)** Solve the dynamic programming problem given these parameters (i.e. compute the functions $\bar{V}_0(a_t)$ and $\bar{V}_1(a_t)$)

**c)** Using  $\bar{V}_0(a_t)$ and $\bar{V}_1(a_t)$, compute the probability of replacement for each $a_t$, i.e. $Prob(i_t =1 | a_t)$

**d)** Comute the likelihood of each observation $j$, e.g. $L_j = i_j Prob(i_j=1|a_j)+(1-i_j)(1-Prob(i_j=1|a_j))$

**e)** Return $-\ln(L)=-\sum_j \ln(L_j)$ (the minus is if you are using a minimization (rather than maximization)procedure)


Explain why you think I've suggested estimation is based on a conditional likelihood (conditional on $a_t$).

**Answer:**

In [490]:
# Data upload

data = np.loadtxt("data.asc")
print(data)

[[ 4.  1.]
 [ 4.  1.]
 [ 3.  0.]
 ..., 
 [ 2.  0.]
 [ 5.  1.]
 [ 2.  1.]]


In [491]:
# Dimensions of the data
np.shape(data)

(1000, 2)

In [492]:
# Age of machine
age = data[:,0]
age = age.reshape(np.size(data,0),1)
# Dummy Variable = 1 if replacement
inv = data[:,1]
inv = inv.reshape(np.size(data,0),1)

In [493]:
df = pd.DataFrame(data, columns=['Age','Replacement'])
df.describe()

,Age,Replacement
count,1000.00000,1000.000000
mean,2.84900,0.580000
std,1.41923,0.493805
min,1.00000,0.000000
25%,2.00000,0.000000
50%,3.00000,1.000000
75%,4.00000,1.000000
max,5.00000,1.000000


In [494]:
def contraction(Vn, theta1, R):
    Vnplusone=np.zeros((10, 1))
    beta = 0.9
    # The first 5 rows correspond to V_0. Ordered by the possible values of a_t: 1,2,3,4,5.
    Vnplusone[0] = theta1*1 + beta*(0.5772+np.log(np.exp(Vn[1])+np.exp(Vn[6])))
    Vnplusone[1] = theta1*2 + beta*(0.5772+np.log(np.exp(Vn[2])+np.exp(Vn[7])))
    Vnplusone[2] = theta1*3 + beta*(0.5772+np.log(np.exp(Vn[3])+np.exp(Vn[8])))
    Vnplusone[3] = theta1*4 + beta*(0.5772+np.log(np.exp(Vn[4])+np.exp(Vn[9])))
    Vnplusone[4] = theta1*5 + beta*(0.5772+np.log(np.exp(Vn[4])+np.exp(Vn[9])))
    #The last 5 rows correspond to V_1
    Vnplusone[5] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[6] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[7] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[8] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    Vnplusone[9] = R + beta*(0.5772+np.log(np.exp(Vn[0])+np.exp(Vn[5])))
    return Vnplusone

In [495]:
def logit(x):
    j = np.exp(x)/(1+np.exp(x))
    return j

In [496]:
def lkhd(i,p):
    # First argument: investment
    # Second argument: probability of replacement
    lkhd = i*p+(1-i)*(1-p)
    return lkhd

In [497]:
# Putting things together to write the objective function

def obj(theta):
    
    theta1= theta[0]
    R = theta[1]
    
    # Initial value for iterations:
    Vn = np.ones((10, 1))
    
    # Finding the fixed point:
    tres = 100
    while (tres > 0.001):
        Vnplusone = contraction(Vn, theta1, R)
        tres = LA.norm(Vnplusone-Vn)
        Vn = Vnplusone
        
    # Computing the probability of replacement for each age:
    Reprob = np.zeros((5,1))

    Reprob[0]=logit(Vn[5]-Vn[0]) # Age = 1
    Reprob[1]=logit(Vn[6]-Vn[1]) # Age = 2
    Reprob[2]=logit(Vn[7]-Vn[2]) # Age = 3
    Reprob[3]=logit(Vn[8]-Vn[3]) # Age = 4
    Reprob[4]=logit(Vn[9]-Vn[4]) # Age = 5
    
    # Matching prob. of replacement to each observation according to age:
    prob=np.zeros((np.size(data,0), 1))

    for j in range(0, np.size(data,0)):
        if age[j] == 1:
            prob[j]= Reprob[0]
        elif age[j] == 2:
            prob[j]= Reprob[1]
        elif age[j] == 3:
            prob[j]= Reprob[2]
        elif age[j] == 4:
            prob[j]= Reprob[3]
        elif age[j] == 5:
            prob[j]= Reprob[4]
            
    # Computing the likelihood of each observation:
    loglkhd = np.log(lkhd(inv,prob))
    
    logL = - np.sum(loglkhd, axis=0)
    #print('Log-likelihood:', logL )
    return logL

In [498]:
# Initial values for numerical optimization
theta =  [-1,-3] 
print(theta)

[-1, -3]


In [499]:
# Parameter estimates
result = minimize(obj, theta, method='nelder-mead')
theta_hat = result.x 
print(theta_hat)

[-1.14838832 -4.44640774]


In [500]:
# Results

print  ( '{:>9} {:>10}  {:>5} '.format('', 'theta1', 'R'))  

print  ( '{:>10} {:>10.5f} {:>10.5f}  '.format('est:', float(theta_hat[0]), float(theta_hat[1]))) 

              theta1      R 
      est:   -1.14839   -4.44641  


* I think you suggested estimation based on a conditional likelihood (conditional on $a_t$) because under the logit error assumption we can solve analytically for the replacement probaility given age.

* Also, there is no individual heterogeneity across buses in this model.

### Question 6
Compute standard errors of your estimates (you should use the regular maximum likelihood standard error formulas).

**Answer:**

In [501]:
# Computation of individual log likelihood

def iloglkhd(beta):
    
    theta1= beta[0]
    R = beta[1]
    
    # Initial value for iterations:
    Vn = np.ones((10, 1))
    
    # Finding the fixed point:
    tres = 100
    while (tres > 0.001):
        Vnplusone = contraction(Vn, theta1, R)
        tres = LA.norm(Vnplusone-Vn)
        Vn = Vnplusone
        
    # Computing the probability of replacement for each age:
    Reprob = np.zeros((5,1))

    Reprob[0]=logit(Vn[5]-Vn[0]) # Age = 1
    Reprob[1]=logit(Vn[6]-Vn[1]) # Age = 2
    Reprob[2]=logit(Vn[7]-Vn[2]) # Age = 3
    Reprob[3]=logit(Vn[8]-Vn[3]) # Age = 4
    Reprob[4]=logit(Vn[9]-Vn[4]) # Age = 5
    
    # Matching prob. of replacement to each observation according to age:
    prob=np.zeros((np.size(data,0), 1))

    for j in range(0, np.size(data,0)):
        if age[j] == 1:
            prob[j]= Reprob[0]
        elif age[j] == 2:
            prob[j]= Reprob[1]
        elif age[j] == 3:
            prob[j]= Reprob[2]
        elif age[j] == 4:
            prob[j]= Reprob[3]
        elif age[j] == 5:
            prob[j]= Reprob[4]
            
    # Computing the likelihood of each observation:
    loglkhd = np.log(lkhd(inv,prob))
    
    return loglkhd

In [502]:
# Variance computation

def var(theta):
    
    theta1= theta[0]
    R = theta[1]
    
    # Numerical computation of derivatives of individual likelihood:
    
    dtheta1 = (iloglkhd([(theta1*1.001),R])-iloglkhd(theta))/(0.001*theta1)
    dR = (iloglkhd([theta1,(R*1.001)] )-iloglkhd(theta))/(0.001*R)
    
    # Creating the variance matrix
    
    variance=np.zeros((np.size(theta), np.size(theta)))
    
    variance[0,0] = dtheta1.T @ dtheta1
    variance[1,1] = dR.T @ dR
    variance[0,1]= dtheta1.T @ dR
    variance[1,0]= variance[0,1]
    
    VAR = np.linalg.inv(variance)
    
    return VAR

In [503]:
# Variance matrix
Var = var(theta_hat)
print(Var)

[[ 0.00560651  0.02083656]
 [ 0.02083656  0.10302951]]


In [504]:
# Results with standard errors

print  ( '{:>9} {:>10}  {:>5} '.format('', 'theta1', 'R'))  

print  ( '{:>10} {:>10.5f} {:>10.5f}  '.format('est:', float(theta_hat[0]), float(theta_hat[1])))

print  ( '{:>10} {:>10.5f} {:>10.5f}  '.format('se:', float(np.sqrt(Var[0,0])), float(np.sqrt(Var[1,1]))))

              theta1      R 
      est:   -1.14839   -4.44641  
       se:    0.07488    0.32098  


### Question 7

Describe (you do NOT have to do this on the computer) how you would need to change your model (either the dynamic programming problem, the estimation procedure, or both) to accommodate the following perturbations of the model:

**a)** Consider an alternative empirical model. Suppose there are two types of firms - differing in their value of $\theta_1$. Proportion $\alpha$ of firms have $\theta_1=\theta_{1A}$, proportion $(1-\alpha)$ have $\theta_1=\theta_{1B}$. How would you change both the dynamic programming problem and the likelihood function? You can ignore the potential initial conditions problem in the likelihood function.

**b)** What if you had the model in a) but you have panel data, i.e. multiple observations on each firm? Write down the likelihood function (again, ignoring potential initial conditions problems).

**c)** Continuing with the panel data situation, what if instead of *firms* differing in $\theta_1$, *machines* differ in $\theta_1$, i.e. when a firm replaces their old machine, the new machine may have $\theta_1=\theta_{1A}$ (w/prob $\alpha$) or it may have $\theta_1=\theta_{1B}$ (w/prob $1-\alpha$)? Again, describe how the dynamic programming problem and likelihoods change
(ignoring potential initial conditions problems)

**d)** Describe the initial conditions problem that you have ignored in the prior three parts. Propose at least one way to address this initial conditions problem.

**e)** Go back to only one "type" of firm and one observation per firm. What if $a_t$ does not evolve deterministically, i.e. if you don't replace ($i_t = 0$):

\begin{equation}
a_{t+1}=
\begin{cases}
&a_t &\quad \text{with probability $\lambda$}\\
&\min\{5, a_t+1\} &\quad \text{if $1-\lambda$}
\end{cases}
\end{equation}

Make an informal argument that there is some information in the data regarding the parameter $\lambda$. Note that this is not obvious because you only have 1 data point per firm and thus you never actually observe transitions from $a_t$ to $a_{t+1}$. You can assume that your data is a random sample of firms that are in a "steady state" (i.e. they have existed for a long time).

### Question 8

Estimate the original model using the Hotz and Miller algorithm. Compare your estimates to those in part 4). You do not need to compute standard errors. If you have time, iterate this estimation procedure (as described in the class notes) and see how many iterations it takes to get "close" to the ML estimator.

**Answer:**

In the specific case where $\epsilon_{0t}$ and $\epsilon_{1t}$ are  assumed Type I Extreme Value (logit) errors, the first equation of the HM alternative representation is given by:

\begin{equation}
\begin{split}
&V(a_t;\theta)=[I-(1-P(a_t; \theta))\beta T(0;\theta)-P(a_t; \theta)\beta T(1;\theta)]^{-1}\\
&[(1-P(a_t; \theta))[u(a_t,0;\theta)+0.5772-\ln(1-P(a_t;\theta))]+P(a_t;\theta)[u(a_t,1;\theta)+0.5772-\ln(P(a_t;\theta))]].
\end{split}
\end{equation}

Also, in this case, the second equation of the HM alternative representation is given by:

\begin{equation}
\begin{split}
P(a_t; \theta)=\frac{\exp(u(a_t,1;\theta)+\beta T(1; \theta)V(a_t;\theta))}{\exp(u(a_t,0;\theta)+\beta T(0; \theta)V(a_t;\theta)+\exp(u(a_t,1;\theta))+\beta T(1; \theta)V(a_t;\theta))}.
\end{split}
\end{equation}

Substituting the first equation into the second then results in something that looks like

\begin{equation}
P(a_t; \theta)= \kappa(P(a_t; \theta), T(1;\theta), T(0;\theta),u(a_t,1;\theta), u(a_t, 0;\theta),\beta),
\end{equation}

where the $\kappa$ function is easily computable.

In [ ]:
# Computing consistent estimates of probabilities of actions